<a href="https://colab.research.google.com/github/sion0507/Robot_simul_code/blob/main/Robot_Test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip -q install pybullet gradio==4.* pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 11.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.34.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 12.0 which is incompatible.
google-adk 1.13.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.
yfinance 0.2.65 requires websockets>=13.0, but you have webso

In [17]:
# <시뮬레이터 전체 초기화(다시 처음부터 하고싶을때 이 셀부터 실행)>
import pybullet as p, pybullet_data

p.resetSimulation()              # 모든 바디/환경 삭제          # 시뮬레이션 타임스텝 재설정


In [18]:
# 1. 모듈 불러오기 및 로봇 식별 아이디 생성, 기본세팅(중력,시간분할 단위, 데이터 path)
import numpy as np
import time,math, re, json
import gradio as gr
from PIL import Image

CID= p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.8)
dt = 1/120
p.setTimeStep(dt)

In [19]:
# 2. 평면 및 장애물 설정

# 2-1) 평면 설정
plane_id = p.loadURDF("plane.urdf")

# 2-2) 기본 박스 만드는 함수 입력=크기,색깔,위치/출력=블록 아이디(int)
def make_box(size=(0.4, 0.4, 0.2), rgba=(0.5,0.5,0.5,1), pos=(1.2,0,0.1)):
  half = [s/2 for s in size]
  vid = p.createVisualShape(p.GEOM_BOX, halfExtents=half, rgbaColor=rgba)
  cid = p.createCollisionShape(p.GEOM_BOX, halfExtents=half)
  bid = p.createMultiBody(baseMass=0, baseCollisionShapeIndex=cid, baseVisualShapeIndex=vid, basePosition=pos)
  return bid

# 2-3) 장애물 생성
obstacles = [] #장애물 블록 아이디 리스트
obstacles.append(make_box((0.6,0.6,0.4), (0.6,0.6,0.6,1), (1.2,0.0,0.2)))
obstacles.append(make_box((0.6,0.6,0.4), (0.6,0.6,0.6,1), (0.5,1.0,0.2)))
obstacles.append(make_box((0.6,0.6,0.4), (0.6,0.6,0.6,1), (0.0,-1.0,0.2)))

blue_cube_id = make_box((0.3,0.3,0.3), (0.1,0.4,1.0,1), (1.5, 0.8, 0.15)) # 로봇이 최종적으로 찾아갈 대상

# 만들어진거 체크
body_count=p.getNumBodies()
print(body_count)


5


In [20]:
# 3. 로봇 본체 만들기(차동구동 근사) + 전방 거리센서(rayTest)


# 3-1) 로봇 본체(원통형)
#  - 시각형상: createVisualShape(..., length=0.1)
#  - 충돌형상: createCollisionShape(..., height=0.1)
#    * 주의: visual은 length, collision은 height라는 이름을 쓴다.

body_vis = p.createVisualShape(p.GEOM_CYLINDER, radius=0.15, length=0.1, rgbaColor=[0.9,0.1,0.1,1])
body_col = p.createCollisionShape(p.GEOM_CYLINDER, radius = 0.15, height = 0.1)
robot_id = p.createMultiBody(baseMass = 5, baseCollisionShapeIndex = body_col, baseVisualShapeIndex = body_vis, basePosition=[0.0,0.0,0.1])


In [ ]:
# 3-2) 로봇 상태 및 동작 정의 클래스
import math, numpy as np

class Robot:
  """
    단순 차동구동 근사:
      - 실제 바퀴/조인트 대신 '선속 v'와 '각속 w'를 시간 dt 동안 적용해 pose를 직접 갱신.
      - 전방 거리센서: rayTest로 안전거리(safe_stop_dist) 내에 장애물이 있으면 선속 0으로 강제.
    """